In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [152]:
#고급 차량 관련 문서로 테스트 진행
doc = df_add[:2]
doc

['Indexation of customs duty rates on excise-equivalent goods on 1 August 2022 australian notice no. 2022/35 indexation of duty rates on excise-equivalent goods on 1 august 2022 this notice sets out the new duty rates for certain excise-equivalent goods, including spirits, beers and fuel products, operative from 1 august 2022. section 19 of the tariff act 1995 (customs tariff act) provides for duty rates for certain excise-equivalent goods to be indexed biannually, in february and august, to the consumer price index (cpi). the new rates are determined by the application of an indexation factor. this indexation factor is calculated by dividing the most recent june or december quarter cpi number by the previous highest june or december quarter cpi number occurring after the june 1983 quarter. the december 2021 quarter cpi figure (121.3) will be used to determine the indexation factor for the duty rate increase applied on 1 august 2022, as it is higher than the previously used june 2021 q

In [153]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

#구두점 제거
doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()
print(len(doc1))

#숫자 제거
doc2 = "".join([i for i in doc1 if not i.isdigit()])
print(len(doc2))

#월 제거
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
         'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   
    
doc3 = " ".join([i for i in doc2.split() if i not in month])
print(doc3)

# word_tokens = word_tokenize(doc2)

#동사 명사
# word_tokens = pos_tag(word_tokenize(doc))
# word_tokens = [t[0] for t in word_tokens if t[1] == "NN"]

5876
5578
Indexation of customs duty rates on excise-equivalent goods on August australian notice no. / indexation of duty rates on excise-equivalent goods on this notice sets out the new duty rates for certain excise-equivalent goods, including spirits, beers and fuel products, operative from . section of the tariff act (customs tariff act) provides for duty rates for certain excise-equivalent goods to be indexed biannually, in and august, to the consumer price index (cpi). the new rates are determined by the application of an indexation factor. this indexation factor is calculated by dividing the most recent or quarter cpi number by the previous highest or quarter cpi number occurring after the quarter. the quarter cpi figure (.) will be used to determine the indexation factor for the duty rate increase applied on , as it is higher than the previously used quarter cpi figure (refer to acn /). on , the australian bureau of statistics released the quarter cpi figure (.). the figures us

In [154]:
n_gram_range = (2, 2)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc3])
candidates = count.get_feature_names_out()

print('bigram 개수 :',len(candidates))
print('bigram 출력 :',candidates)

bigram 개수 : 375
bigram 출력 : ['aac including' 'abf arrange' 'abf gov' 'acn australian' 'act australian'
 'act customs' 'act excise' 'act provides' 'additional duty'
 'additional note' 'alcoholexciserates ato' 'alterations notice'
 'amendment cost' 'amendments online' 'application factor'
 'application indexation' 'applied higher' 'applied implementation'
 'applies goods' 'apply goods' 'apply period' 'apply refund'
 'apply tariff' 'arrange publication' 'arrangements outlined'
 'asean australia' 'ato gov' 'ato make' 'ato website' 'attachment notice'
 'attachment online' 'attachment removal' 'au alcoholexciserates'
 'au amendments' 'au fuelexciserates' 'au reflect' 'august australian'
 'august consumer' 'australia new' 'australia service' 'australian border'
 'australian bureau' 'australian notice' 'australian taxation'
 'available attachment' 'available commencement' 'available current'
 'available www' 'beers fuel' 'belarus currently' 'biannually august'
 'border force' 'bureau statistic

In [155]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc]) #전체 문서
candidate_embeddings = model.encode(candidates)#추출한 bigram

In [156]:
#문서와 가장 유사한 키워드 추출
doc_bigram_keywords = []

top_n = 20
distances = cosine_similarity(doc_embedding, candidate_embeddings)
doc_bigram_keyword = [candidates[index] for index in distances.argsort()[0][-top_n:]]
doc_bigram_keywords.append(doc_bigram_keyword)

print(len(doc_bigram_keywords))
print(doc_bigram_keywords)

1
[['tax revenue', 'au fuelexciserates', 'tariff increase', 'new tariff', 'vehicles australian', 'tariff amendment', 'online tariff', 'goods australia', 'tariff subheadings', 'spirits beers', 'rates excise', 'revised tariff', 'august australian', 'tariff alterations', 'indirect tax', 'fuelexciserates fuel', 'car tax', 'asean australia', 'tax threshold', 'australian taxation']]


In [157]:
#문서와 가장 유사한 키워드 벡터화
doc_bigram_embedding = []
for keyword in doc_bigram_keywords:
    doc_bigram_embedding.append(model.encode(keyword))
print(len(doc_bigram_embedding))

# ['신규 변경', '신규 관세', '자동차세', '온라인 관세', '세금 임계값']

1


In [158]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [147]:
#doc_bigram_keywords -> doc_bigram_embedding
#keyword -> keyword_embedding

In [150]:
#문서 키워드(doc_bigram_embedding)와 키워드 100(keyword_embedding) 유사도 비교 
bigram_key_result = []
keyword_key_result = []


for index, bigram in enumerate(doc_bigram_embedding):
    bigram_result = []
    key_result=[]
    
    for i in range(len(bigram)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([bigram[i]],[keyword_embedding[j]])
            if distances[0][0] > 0.70:
                bigram_result.append(doc_bigram_keywords[index][i])
                key_result.append(keyword[j])
#         print(bigram_result)
#         print(key_result)
    bigram_key_result.append(set(bigram_result))
    keyword_key_result.append(set(key_result))

In [151]:
df_check = pd.DataFrame()

print(bigram_key_result)
print(keyword_key_result)
# bigram = pd.Series(bigram_key_result)
# key = pd.Series(keyword_key_result)

# bigram = bigram.to_frame()
# bigram


[{'fuelcell vehicles', 'hybrid vehicles', 'tariff available', 'car tax', 'hydrogen fuelcell', 'vehicles hydrogen', 'removal customs', 'duty electric'}]
[{'gourd', 'machinery', 'copper', 'car', 'lubricating oil', 'LNG (liquid natural gas)', 'propane', 'feed', 'molasses', 'Automotive Parts', 'beverage', 'Liquid pump'}]
